# Sky Background Magnitude Uncertainties

We're going to perform the aperture photometry. This technique consists in calculate all the flux $F$ on a determined aperture, e.g, measure all the light of the pixels inside the aperture. This flux will have $[adu]$ units. In our case, we're going to use circular apertures with a radius equals to 5 pixels distributed throughout the sky background, approximately 120 apertures per CCD.

Then, we have to calculate magnitudes. The magnitude is calculated in this way:

\begin{equation*}
    m=ZP-2.5\log_{10}{\left(F\right)}
\end{equation*}

where $ZP$ is the photometric "zero point" (meanwhile, we consider $ZP=0$; however, the ZP doesn't affect to the uncertainties calculation) and $F$ is the total flux in the aperture in units of $[adu]$. We'll have 120 instrumental magnitudes measured on the sky background. Now, we want the fluctuation of this magnitudes, so we're going to calculate the standard deviation of the magnitudes. The formula is:

\begin{equation*}
    \sigma = \sqrt{\frac{1}{N}\displaystyle\sum_{i}^{N}(m_i-\bar{m})^2}
\end{equation*}

where $N=120$ is the total of apertures, $m_i$ is the magnitude on the aperture $i$ and $\bar{m}$ is the mean magnitude of the sample. Now, the quantity $\sigma$ is unique per CCD (since it considers fluctuations caused by the effects of that CCD) and needs to be added in quadrature-sum to the magnitude error calculated by SEXtractor while performs the photometry. Considering $\delta m$ as the magnitude error of a source, now the total magnitude error is defined by:

\begin{equation*}
    \Delta m_j^2=\delta m_j^2+\sigma^2 \Longrightarrow \Delta m_j=\sqrt{\delta m_j^2+\sigma^2}
\end{equation*}

where $j$ is the source in a CCD. Therefore, we have to calculate a $\sigma^2$ fluctuation for every CCD (and filter too) caused by pupil ghosts, fringing, combine residuals, etc. Then, the actual magnitude error formula is:

\begin{equation*}
    \Delta m_{jk}^2=\delta m_{jk}^2+\sigma_j^2 \Longrightarrow \Delta m_{jk}=\sqrt{\delta m_{jk}^2+\sigma_j^2}
\end{equation*}

where now $j$ is the CCD and $k$ is the source in that CCD.

In [5]:
from astropy.io import fits #import FITS images to the notebook
import numpy as np #numpy, to open files and maths
from photutils.aperture import CircularAperture,aperture_photometry #to create apertures and perform photometry
import os #to change directories

## Background Uncertainties Function

In [6]:
def back_uncertainties(decam_filter,ccd):
    os.chdir('/home/alex-cfrd/Escritorio/SEXtractor/filter_'+decam_filter+'/background_points') #move to directory
    SIGMA = [] #to allocate sigma for each ccd
    for i in range(1,32):
        if i==30 and ccd=='N': #ignore ccd [N30]
            continue
        elif i==7 and ccd=='S':
            continue
        image = fits.open('/home/alex-cfrd/Escritorio/SEXtractor/filter_'+decam_filter+'/back_'+ccd+decam_filter+'.fits')[ccd+str(i)] #load image
        data = image.data #extract image data only
        positions = np.genfromtxt(ccd+str(i)+'_'+decam_filter+'.coo') #load coordinates files
        apertures = CircularAperture(positions,r=10) #convert coordinates into apertures
        phot_table = aperture_photometry(data,apertures,method='subpixel',subpixels=5) #perform aperture photometry
        flux = phot_table['aperture_sum'] #extract flux sum column
        #mag = -2.5*np.log10(flux) #calculate instrumental magnitud of all the apertures
        sigma = np.std(flux) #calculate std deviation of the 120 apertures
        SIGMA.append(sigma) #append ccd sigma to SIGMA list
        if i<10:
            print(str(sigma)[:9])#'Filter '+decam_filter+' sigma ['+ccd+str(i)+']  =',sigma)
        else:
            print(str(sigma)[:9])#'Filter '+decam_filter+' sigma ['+ccd+str(i)+'] =',sigma)
    #STATISTICS
    print('---------------------------------------------')
    print('STATISTICS')
    mean = str(np.mean(SIGMA)) #mean of S CCDs sigmas
    #if ccd=='S':
    #    ind_med = ' ['+ccd+str(SIGMA.index(np.median(SIGMA))+1)+']' #calculate median sigma
    median = str(np.median(SIGMA)) #calculate median sigma
    minimum,ind_min = str(np.min(SIGMA)),' ['+ccd+str(SIGMA.index(np.min(SIGMA))+1)+']' #extract min sigma
    maximum,ind_max = str(np.max(SIGMA)),' ['+ccd+str(SIGMA.index(np.max(SIGMA))+1)+']' #extract max sigma
    std = str(np.std(SIGMA)) #calculate sigmas standard deviation
    suma = str(sum(SIGMA))
    print('---------------------------------------------')
    print('SIGMA Mean   =',mean[:9])
    print('SIGMA Median =',median[:9])
    print('SIGMA Min    =',minimum[:9]+ind_min)
    print('SIGMA Max    =',maximum[:9]+ind_max)
    print('SIGMA Std    =',std[:9])
    print('SIGMA Sum    =',suma[:9])

## Filter $Y$ [S1-S31]

In [7]:
back_uncertainties('Y','S')

32622.293
2125.4919
1939.5668
1765.3545
1841.3462
143222.47
2516.7406
2133.2772
1907.6856
2044.5732
1415.5726
3508.3239
1858.9496
1966.4349
2175.7769
1762.5692
19277.552
81128.484
25565.593
1968.7605
1933.3357
1809.6753
21947.775
14088.961
2485.0583
1923.8790
8609.6188
23604.198
14634.112
17500.153
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 14709.453
SIGMA Median = 2154.5270
SIGMA Min    = 1415.5726 [S11]
SIGMA Max    = 143222.47 [S6]
SIGMA Std    = 28640.610
SIGMA Sum    = 441283.59


## Filter $Y$ [N1-N31] (no [N30])

In [8]:
back_uncertainties('Y','N')

1786.2876
1824.9599
2139.2491
1719.1970
1706.5422
1984.1148
2048.4604
1920.3171
1421.6604
1903.9503
1816.5612
2051.3599
2392.5632
2032.3630
1682.8309
1634.2545
1776.2431
1733.1582
2264.4709
1925.8111
1473.8794
1890.9768
1668.7958
1714.7385
1929.6671
1650.4255
2786.7164
1674.6157
1682.0176
1449.9942
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 1856.2061
SIGMA Median = 1801.4244
SIGMA Min    = 1421.6604 [N9]
SIGMA Max    = 2786.7164 [N27]
SIGMA Std    = 279.93297
SIGMA Sum    = 55686.183


## Filter $u$ [S1-S31] (no [S7])

In [9]:
back_uncertainties('u','S')

77.600408
52.883706
29.827814
77.712623
53.766727
25.996806
135.00006
64.123322
70.954445
26.852417
45.415931
30.641896
30.917956
39.464379
35.832179
73.084019
67.696212
28.144978
76.476152
66.592945
51.077395
42.695970
90.433026
31.303542
56.773276
34.555774
42.600078
40.469870
95.633523
34.496589
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 54.300801
SIGMA Median = 48.246663
SIGMA Min    = 25.996806 [S6]
SIGMA Max    = 135.00006 [S7]
SIGMA Std    = 24.861654
SIGMA Sum    = 1629.0240


## Filter $u$ [N1-N31] (no [N30])

In [10]:
back_uncertainties('u','N')

121.37240
93.652845
120.12744
46.645432
37.511536
52.100536
22.518595
30.219789
47.458005
103.91576
33.525058
86.584458
64.295233
131.66227
28.344850
38.690151
30.589835
25.848131
104.23203
46.899635
37.517616
49.549433
46.916857
99.128044
26.696899
33.176492
48.468976
32.730062
34.536719
127.42518
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 60.078010
SIGMA Median = 46.908246
SIGMA Min    = 22.518595 [N7]
SIGMA Max    = 131.66227 [N14]
SIGMA Std    = 34.656427
SIGMA Sum    = 1802.3403


## Filter $g$ [S1-S31] (no [S7])

In [11]:
back_uncertainties('g','S')

339.86629
298.45061
176.18324
470.67144
838.68454
139.84994
1174.7456
315.54012
146.07054
111.29942
471.53296
146.31982
196.57432
338.08628
289.22371
171.63545
107.04028
250.85757
136.38952
175.44880
429.70910
127.69577
509.68576
193.69246
1220.8833
169.00027
163.18453
260.06340
1542.5794
223.02990
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 371.13315
SIGMA Median = 236.94374
SIGMA Min    = 107.04028 [S17]
SIGMA Max    = 1542.5794 [S29]
SIGMA Std    = 352.88001
SIGMA Sum    = 11133.994


## Filter $g$ [N1-N31] (no [N30])

In [12]:
back_uncertainties('g','N')

96.662262
251.64435
1838.9072
415.68211
434.71001
814.68081
169.68542
265.59605
544.18494
212.18400
131.78101
114.47600
450.80122
175.08513
194.84582
681.83823
332.66902
381.70381
262.25154
267.83089
568.27095
236.43404
206.24061
91.441297
316.96220
346.25337
1554.9275
219.43511
526.89310
96.039437
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 406.67058
SIGMA Median = 266.71347
SIGMA Min    = 91.441297 [N24]
SIGMA Max    = 1838.9072 [N3]
SIGMA Std    = 388.40490
SIGMA Sum    = 12200.117


## Filter $r$ [S1-S31] (no [S7])

In [13]:
back_uncertainties('r','S')

742.21891
439.72043
364.55230
638.47727
1644.8730
222.56763
1163.3281
253.33073
222.40224
251.92534
268.07076
197.12442
391.53334
505.32296
264.65241
261.36492
217.80469
562.45226
508.55658
244.85162
624.28849
222.44441
461.91182
305.73339
386.22822
230.58152
222.38936
371.39784
825.05206
509.91029
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 450.83558
SIGMA Median = 367.97507
SIGMA Min    = 197.12442 [S12]
SIGMA Max    = 1644.8730 [S5]
SIGMA Std    = 310.34369
SIGMA Sum    = 13525.067


## Filter $r$ [N1-N31] (no [N30])

In [14]:
back_uncertainties('r','N')

235.47671
565.26419
6873.0888
1109.3096
606.78211
3039.0622
321.91034
446.42197
458.08568
443.09666
304.35343
220.36343
2214.7166
278.66206
473.85672
1128.6504
570.58697
1232.6967
9090.3829
327.28872
916.85360
367.34045
315.73743
245.31821
721.97460
388.13892
4051.7296
442.33609
1201.7998
1218.5487
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 1326.9944
SIGMA Median = 519.56045
SIGMA Min    = 220.36343 [N12]
SIGMA Max    = 9090.3829 [N19]
SIGMA Std    = 1990.7186
SIGMA Sum    = 39809.834


## Filter $z$ [S1-S31] (no [S7])

In [15]:
back_uncertainties('z','S')

1048.4459
1833.8201
2428.8501
2078.8305
1223.1585
1084.6522
8059.0043
933.43272
1807.6995
1258.5427
1002.6989
1560.0797
1045.5470
1029.5559
1042.1747
1112.3176
1146.6576
899.28148
1025.6083
1122.6600
2010.5801
1639.7117
1117.4068
1101.2067
2487.1857
1032.9107
1241.9626
89018.862
113083.22
28557.046
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 9134.4373
SIGMA Median = 1184.9080
SIGMA Min    = 899.28148 [S18]
SIGMA Max    = 113083.22 [S29]
SIGMA Std    = 25260.579
SIGMA Sum    = 274033.11


## Filter $z$ [N1-N31] (no [N30])

In [16]:
back_uncertainties('z','N')

1040.5633
2501.2521
6623.7406
3654.0828
1620.7585
27199.987
1051.3923
3681.4028
1461.1630
2121.1649
1169.0034
951.21328
8438.1995
1194.0316
1539.3470
8053.4061
1624.7313
5343.4549
17361.006
1716.5592
2468.5963
1069.6719
5190.4213
1103.2094
1435.3902
5427.9707
9540.4458
2630.3878
88117.757
95423.592
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 10358.463
SIGMA Median = 2484.9242
SIGMA Min    = 951.21328 [N12]
SIGMA Max    = 95423.592 [N30]
SIGMA Std    = 22451.667
SIGMA Sum    = 310753.90


## Filter $i$ [S1-S31] (no [S7])

In [17]:
back_uncertainties('i','S')

975.64785
1179.6035
927.73576
1019.7208
1873.1531
665.56354
2472.3669
893.10032
795.89496
1076.8611
842.44530
788.51172
1014.6321
809.31124
1019.6632
849.17825
873.79834
783.95858
896.81171
967.90919
1212.8023
939.28469
1316.3244
824.82579
835.32043
750.57868
892.77961
784.72724
1467.7590
847.23200
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 1019.9167
SIGMA Median = 894.95602
SIGMA Min    = 665.56354 [S6]
SIGMA Max    = 2472.3669 [S7]
SIGMA Std    = 360.56562
SIGMA Sum    = 30597.502


## Filter $i$ [N1-N31] (no [N30])

In [18]:
back_uncertainties('i','N')

741.77070
1112.0998
1006.3825
6875.6399
4229.3782
11020.247
814.35508
877.30942
2914.8579
843.89124
971.75161
789.78680
1873.6615
961.26644
777.66875
28818.520
927.81958
3750.8702
12138.167
1144.6378
2147.2596
1237.8827
889.64515
778.32826
905.94820
914.30246
14459.449
848.24564
1419.1388
26792.782
---------------------------------------------
STATISTICS
---------------------------------------------
SIGMA Mean   = 4432.7688
SIGMA Median = 1059.2411
SIGMA Min    = 741.77070 [N1]
SIGMA Max    = 28818.520 [N16]
SIGMA Std    = 7188.2196
SIGMA Sum    = 132983.06
